<a href="https://colab.research.google.com/github/vprobon/iLIR-ML-data/blob/main/utilities/LIR_in_AFhuman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

In [ ]:
import pandas as pd

AF_dir = "/home/vprobon/AlphaFoldLIRs/AlphaFold_Human_06042022" # Directory in cluster


def residueb(residue):
  totalb = 0
  count=0
  for atom in residue:
    totalb += atom.bfactor
    count += 1
  return(totalb/count)

def process_LIRs(structure):
  residues = [r for r in structure.get_residues() if r.get_id()[0] == " "]
  for res in residues:
    if res.id[1] == len(residues)-2:
      # Can't find a core LIR from now on
      return
    if res.resname in ['TRP', 'TYR', 'PHE']:
      aroma_id = res.id[1]-1

      if residues[aroma_id+3].resname in ['VAL','LEU','ILE']:
        ## Then we have a LIR-motif here
        print(f"{structure.id}\t{aroma_id+1:6d}\t{aroma_id+4:6d}", end="\t")
        allb=[]
        upstreamb=[]
        for i in range (-10,0):
          #print(residues[aroma_id+i].resname, residueb(residues[aroma_id+i]))
          if aroma_id+i < 0:
            continue
          upstreamb.append(residueb(residues[aroma_id+i]))
          allb.append(residueb(residues[aroma_id+i]))
        #print()
        lirb=[]
        for i in range (0,4):
          #print(residues[aroma_id+i].resname, residueb(residues[aroma_id+i]))
          lirb.append(residueb(residues[aroma_id+i]))
          allb.append(residueb(residues[aroma_id+i]))
        #print()
        downstreamb=[]
        for i in range (4,14):
          if aroma_id+i == len(residues):
            break
          #print(residues[aroma_id+i].resname, residueb(residues[aroma_id+i]))
          downstreamb.append(residueb(residues[aroma_id+i]))
          allb.append(residueb(residues[aroma_id+i]))

        if upstreamb: # there is an updstream sequence
          #print("Upstream: ", sum(upstreamb)/len(upstreamb), end = "\t")
          print(f"Up: {sum(upstreamb)/len(upstreamb):.6}", end="\t")
        else:
          print("Up: ", "N/A", end = "\t")

        #print("LIR:", sum(lirb)/len(lirb), end = "\t")
        print(f"LIR: {sum(lirb)/len(lirb):.6}", end = "\t")

        if downstreamb: # there is an updstream sequence
          #print("Downstream", sum(downstreamb)/len(downstreamb))
          print(f"Down: {sum(downstreamb)/len(downstreamb):.6}", end="\t")
        else:
          print("Down", "N/A", end="\t")

        print(f"Overall: { (sum(downstreamb)+ +sum(lirb)+sum(upstreamb))/(len(upstreamb)+len(lirb)+len(downstreamb)):.6} " )

def getAFstructures(AFdir):
  from Bio.PDB.PDBParser import PDBParser
  parser = PDBParser()
  import os
  AFfiles=os.listdir(AFdir)
  structs=[]
  for AFfile in AFfiles:
    if AFfile[-4:] != '.pdb':
      continue
    structure = parser.get_structure(AFfile, AFdir+"/"+AFfile)
    structs.append(structure)
  return structs


if __name__ == '__main__':

  structures = []
  debug = True
  if debug:
    structures = getAFstructures('./dummyAF') # For testing and debugging
  else:
    structures = getAFstructures(AF_dir)
